# Regresión Lineal: El Camino TensorFlow

Para este ejercicio, presentamos cómo realizar una regresión lineal en el contexto de TensorFlow.

Resolveremos el sistema de ecuaciones lineales:

$$ y = Ax + b$$

Con la longitud del Sepal (y) y el ancho del Petal (x) de los datos de Iris.

Realizar una regresión lineal en TensorFlow es mucho más fácil que tratar de entender las descomposiciones de álgebra o matriz lineal para las dos recetas anteriores. Haremos lo siguiente:

 1. Crea la salida del gráfico computacional de regresión lineal. Esto significa que aceptaremos una entrada,  $x$, y generamos la salida, $Ax + b$.
 2. Creamos una función de pérdida, la pérdida L2, y usamos esa salida con la tasa de aprendizaje para calcular los gradientes de las variables del modelo, $A$ y $b$ para minimizar la pérdida.
 
La ventaja de utilizar TensorFlow de esta manera es que el modelo se puede actualizar y ajustar de forma rutinaria con nuevos datos de manera incremental con cualquier tamaño de lote razonable. Cuanto más iterativos hacemos nuestros algoritmos de aprendizaje automático, mejor.

Comenzamos cargando las bibliotecas necesarias.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops
ops.reset_default_graph()

Creamos la sesión del grafo.

In [ ]:
sess = tf.Session()

A continuación, cargamos los datos de Iris de la biblioteca Scikit-Learn.

In [ ]:
# Cargar los datos
# iris.data = [(Sepal Length, Sepal Width, Petal Length, Petal Width)]
iris = datasets.load_iris()
x_vals = np.array([x[3] for x in iris.data])
y_vals = np.array([y[0] for y in iris.data])

Con la mayoría de los algoritmos de TensorFlow, deberemos declarar un tamaño de lote para los marcadores de posición y las operaciones en el gráfico. Aquí, lo establecemos en 25. Podemos establecerlo en cualquier número entero entre 1 y el tamaño del conjunto de datos.

In [ ]:
# Declarar el tamaño del lote (batch)
batch_size = 25

Ahora inicializamos los marcadores de posición y las variables en el modelo.

In [ ]:
# Iniciar los marcadores de posición
x_data = tf.placeholder(shape=[None, 1], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# Crear las variables para la regresión lineal
A = tf.Variable(tf.random_normal(shape=[1,1]))
b = tf.Variable(tf.random_normal(shape=[1,1]))

Añadimos las operaciones del modelo (salida del modelo lineal) y la pérdida de L2.

In [ ]:
# Declarar las operaciones del modelo
model_output = tf.add(tf.matmul(x_data, A), b)

# Declarar la función de pérdida (L2 loss)
loss = tf.reduce_mean(tf.square(y_target - model_output))

Tenemos que decirle a TensorFlow cómo optimizar y volver a propagar los gradientes. Lo hacemos con el operador estándar de descenso de gradiente (`tf.train.GradientDescentOptimizer`), con el argumento de la tasa de aprendizaje de $0.05$.

Luego inicializamos todas las variables del modelo.

In [ ]:
# Declarar el optimizador
my_opt = tf.train.GradientDescentOptimizer(0.05)
train_step = my_opt.minimize(loss)

# Initializar las variables
init = tf.global_variables_initializer()
sess.run(init)

Comenzamos nuestro ciclo de entrenamiento y ejecutamos el optimizador para 100 iteraciones.

In [ ]:
# Ciclo de entrenamiento
loss_vec = []
for i in range(100):
    rand_index = np.random.choice(len(x_vals), size=batch_size)
    rand_x = np.transpose([x_vals[rand_index]])
    rand_y = np.transpose([y_vals[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(temp_loss)
    if (i+1)%25==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)) + ' b = ' + str(sess.run(b)))
        print('Loss = ' + str(temp_loss))

Extraemos los coeficientes óptimos y obtenemos la mejor línea de ajuste.

In [ ]:
# Obtener los coeficientes óptimos
[slope] = sess.run(A)
[y_intercept] = sess.run(b)

# Obtener la mejor línea de ajuste
best_fit = []
for i in x_vals:
  best_fit.append(slope*i+y_intercept)

Grafica los resultados con Matplotlib. Junto con el ajuste lineal, también trazaremos la pérdida de L2 sobre las iteraciones de entrenamiento del modelo.

In [ ]:
# Graficar el resultado
plt.plot(x_vals, y_vals, 'o', label='Data Points')
plt.plot(x_vals, best_fit, 'r-', label='Best fit line', linewidth=3)
plt.legend(loc='upper left')
plt.title('Sepal Length vs Petal Width')
plt.xlabel('Petal Width')
plt.ylabel('Sepal Length')
plt.show()

# Graficar la pérdida con el tiempo
plt.plot(loss_vec, 'k-')
plt.title('L2 Loss per Generation')
plt.xlabel('Generation')
plt.ylabel('L2 Loss')
plt.show()